In [24]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
import cv2
import time

In [25]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

## Loading the model

Models with Keras

In [26]:
def select_model(model):
    models = ['keras_model.h5', 'model_upb.h5', 'model_upb_2.h5', 'model_upb_3.h5', 'model_upb_4.h5']
    return load_model(models[model - 1], compile=False) 

In [27]:
def config_img(model, img):
    if model == 1:
        img = cv2.resize(img, (224, 224))
        img = img.reshape(1, 224, 224, 3)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (50, 50))
        img = img.reshape(1, 50, 50, 1)
    return img

In [10]:
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
font = cv2.FONT_HERSHEY_COMPLEX

model_chosen = 4

model = select_model(model_chosen)

class_names = {
     0: 'ALEJANDRA',
     1: 'MAURICIO',
     2: 'SEBASTIAN',
     3: 'IGNACIO'
}

while True:
    success, img_original = cap.read()
    faces = facedetect.detectMultiScale(img_original, 1.3, 5)
    for x, y, w, h in faces:
        crop_img = img_original[y:y+h, x:x+w]
        img = config_img(model_chosen, crop_img)
        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        probability = np.amax(prediction)

        cv2.rectangle(img_original, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img_original, class_names[class_index], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        cv2.putText(img_original, str(int(probability*100))+"%", (x+w+20, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    try:
        cv2.imshow('Modelo '+str(model_chosen), img_original)
    except:
        pass

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key in [ord('1'), ord('2'), ord('3'), ord('4')]:
        model_chosen = int(chr(key))
        model = select_model(model_chosen)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2023-11-06 12:06:19.433057: W tensorflow/c/c_api.cc:300] Operation '{name:'conv2d_34_2/bias/Assign' id:22868 op device:{requested: '', assigned: ''} def:{{{node conv2d_34_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_34_2/bias, conv2d_34_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-06 12:06:21.724297: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_35_2/Softmax' id:22959 op device:{requested: '', assigned: ''} def:{{{node dense_35_2/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_35_2/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or c

-1

In [28]:
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

class_names = {
     0: 'ALEJANDRA',
     1: 'MAURICIO',
     2: 'SEBASTIAN',
     3: 'IGNACIO'
}

model_chosen = 5
model = select_model(model_chosen)

predicted_person = None
start_time = None
confirmation_duration = 2000
previous_predicted_person = None

while True:
    success, img_original = cap.read()
    faces = facedetect.detectMultiScale(img_original, 1.3, 5)

    for x, y, w, h in faces:
        crop_img = img_original[y:y+h, x:x+w]
        img = config_img(model_chosen, crop_img)
        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        predicted_person = class_names[class_index]
        probability = np.amax(prediction)

        cv2.rectangle(img_original, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img_original, class_names[class_index], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(img_original, str(int(probability*100))+"%", (x+w+20, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        if predicted_person:
            if predicted_person != previous_predicted_person:
                start_time = time.time()
            elif time.time() - start_time >= confirmation_duration / 1000:
                
                banner = img_original.copy()
                cv2.putText(banner, f"Eres {predicted_person}? (S/N)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.imshow('Confirmacion', banner)
                
                while True:
                    key = cv2.waitKey(1) & 0xFF

                    if key == ord('s'):
                        print(f"Respondiste 'si' a {predicted_person}")
                        previous_predicted_person = None
                        break
                    elif key == ord('n'):
                        print(f"Respondiste 'no' a {predicted_person}")
                        previous_predicted_person = None
                        break

                cv2.destroyWindow('Confirmacion')
                continue
        
        previous_predicted_person = predicted_person

    cv2.imshow('Modelo ' + str(model_chosen), img_original)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key in [ord('1'), ord('2'), ord('3'), ord('4')]:
        model_chosen = int(chr(key))
        model = select_model(model_chosen)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


2023-11-06 13:03:25.113100: W tensorflow/c/c_api.cc:300] Operation '{name:'conv2d_41/bias/Assign' id:78528 op device:{requested: '', assigned: ''} def:{{{node conv2d_41/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_41/bias, conv2d_41/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-06 13:03:29.370059: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_41/Softmax' id:78597 op device:{requested: '', assigned: ''} def:{{{node dense_41/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_41/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new se

Respondiste 'si' a MAURICIO
Respondiste 'no' a IGNACIO
Respondiste 'si' a MAURICIO
Respondiste 'si' a MAURICIO


-1